# Практикум по парсингу и работе с API


Булыгин Олег  

* [LinkedIn](linkedin.com/in/obulygin)  
* [Telegram](https://t.me/obulygin91)  
* [Vk](vk.com/obulygin91)  
* email: obulygin91@ya.ru  

[Сообщество по Python](https://yandex.ru/q/loves/pythontalk/) на Кью  
[Сообщество по Data Science и анализу данных](https://yandex.ru/q/loves/datatalk/) на Кью 

## Поработаем с блогом SkillFactory

In [ ]:
import pandas as pd
import requests

In [ ]:
# метод get
res = requests.get('https://blog.skillfactory.ru/')
# res
res.status_code

In [ ]:
# браузер отрисовал бы страницу на основе данного текста
res.text

In [ ]:
# cформируем поисковый запрос, обратите внимание на его формат
URL = 'https://blog.skillfactory.ru/?s=python'

In [ ]:
req = requests.get(URL)
req

In [ ]:
req.text

In [ ]:
# в request можно передать параметры запроса (для get), данные (для post) и заголовки (headers) в виде словарей. 
# сегодня не будем рассматривать примеры с необходимостью передачи заголовка, 
# но в практике вам это точно понадобится
URL = 'https://blog.skillfactory.ru/'
params = {
    's': 'python'
}

req = requests.get(URL, params=params)

req.text

In [ ]:
# как разбирать всю эту разметку? Поможет BeautifulSoup.
# Документация: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup

In [ ]:
# создаем объекта, через методы которого будем искать нужные теги и извлекать их содержимое
soup = BeautifulSoup(res.text)
soup

In [ ]:
# функция finda_all позволяет найти все указанные тег с нужными атрибутами (с вложениями), возвращает список
news = soup.find_all('div', class_='post-card')
news

In [ ]:
for el in news:
    # функция find ищет один указанный тег, возвращает именно его
    title = el.find('div', 'post-card__title').text # поле text позволяет извлечь текстовое содержимое тега
    print(title)
    link = el.find('a').get('href') # метод get позволяет извлечь значение атрибута
    print(link)
    category = el.find('span', 'post-card__category').text
    print(category)
    date = pd.to_datetime(el.find('meta', itemprop='datePublished').get('content'))
    print(date)
    print()

Напишем функцию на основе всех предыдущих действий, которая будет возвращать датафрейм с датой, заголовком, категорией и ссылкой на полный текст поста


In [ ]:
def get_sf_blog_posts(url, query):
    params = {
        's': query
    }
    req = requests.get(url, params=params).text
    soup = BeautifulSoup(req)

    news = soup.find_all('div', class_='post-card')

    sf_blog = pd.DataFrame()

    for el in news:
        title = el.find('div', 'post-card__title').text
        link = el.find('a').get('href') 
        category = el.find('span', 'post-card__category').text
        date = pd.to_datetime(el.find('meta', itemprop='datePublished').get('content')) 
        row = {'date': date, 'title': title, 'link': link, 'category':category}
        sf_blog = pd.concat([sf_blog, pd.DataFrame([row])]) 
    return sf_blog.reset_index(drop=True)

get_sf_blog_posts('https://blog.skillfactory.ru/', 'python')

А как извлечь посты с нескольких страниц?

Для начала напишем функцию для определения количества страниц

In [ ]:
def get_number_of_pages(url, query):
    params = {
        's': query
    }
    req = requests.get(url, params=params).text
    soup = BeautifulSoup(req)
    number_of_pages = int(soup.find_all('a', class_='page-numbers')[-2].text)
    return number_of_pages

print(get_number_of_pages('https://blog.skillfactory.ru/', 'python'))

In [ ]:
import time 

In [ ]:

def get_sf_blog_posts(url, query, pages='all'):
    params = {
        's': query
    }

    if pages == 'all':
        number_of_pages = get_number_of_pages(url, query) # определяем количество страниц для конкретного запроса
        time.sleep(0.2)  # очень часто сайты могут ограничивать частые запросы к себе, поэтому нужно задерживать исполнение
    else:
        number_of_pages = pages

    
    sf_blog = pd.DataFrame()
    for page in range(1, number_of_pages+1):
        page_url = url + f'page/{page}/'
        print(page_url)
        req = requests.get(page_url, params=params).text
        time.sleep(0.3)
        soup = BeautifulSoup(req)

        news = soup.find_all('div', class_='post-card')


        for el in news:
            title = el.find('div', 'post-card__title').text
            # print(title)
            link = el.find('a').get('href') 
            category = el.find('span', 'post-card__category').text
            date = pd.to_datetime(el.find('meta', itemprop='datePublished').get('content')) # вот тут можем получить ошибку 
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            sf_blog = pd.concat([sf_blog, pd.DataFrame([row])]) 

    return sf_blog.reset_index(drop=True)

get_sf_blog_posts('https://blog.skillfactory.ru/', 'python')

In [ ]:
# исправим ситуацию с датой

def get_sf_blog_posts(url, query, pages='all'):
    params = {
        's': query
    }

    if pages == 'all':
        number_of_pages = get_number_of_pages(url, query) # определяем количество страниц для конкретного запроса
        time.sleep(0.2) 
    else:
        number_of_pages = pages
    
    sf_blog = pd.DataFrame()
    for page in range(1, number_of_pages+1):
        page_url = url + f'page/{page}/'
        # print(page_url)
        req = requests.get(page_url, params=params).text
        time.sleep(0.3)
        soup = BeautifulSoup(req)

        news = soup.find_all('div', class_='post-card')


        for el in news:
            title = el.find('div', 'post-card__title').text
            print(title)
            link = el.find('a').get('href') 
            category = el.find('span', 'post-card__category').text
            try:
                date = pd.to_datetime(el.find('meta', itemprop='datePublished').get('content')) 
            except:
                date = pd.NaT
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            sf_blog = pd.concat([sf_blog, pd.DataFrame([row])]) 

    return sf_blog.reset_index(drop=True)

res = get_sf_blog_posts('https://blog.skillfactory.ru/', 'python')
res

А что если мы хотим еще получать полные тексты всех статей? Давайте напишем дополнительную функцию

In [ ]:
# протестируем на двух страницах, чтобы долго не ждать

res = get_sf_blog_posts('https://blog.skillfactory.ru/', 'python', 2)
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        # print(el)
        req = requests.get(el).text
        time.sleep(0.3)
        soup = BeautifulSoup(req)
        full_text = soup.find('div', class_='alignfull').text.strip() # опять получим ошибку
        posts_df.loc[i, 'text'] = full_text
        i += 1
    return posts_df

add_full_text(res)

In [ ]:
res = get_sf_blog_posts('https://blog.skillfactory.ru/', 'python', 2)
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        # print(el)
        req = requests.get(el).text
        time.sleep(0.3)
        soup = BeautifulSoup(req)
        try:
            full_text = soup.find('div', class_='alignfull').text.strip()
        except:
            full_text = soup.find('div', class_='left').text.strip()
        posts_df.loc[i, 'text'] = full_text
        i += 1
    return posts_df

add_full_text(res)

## Примеры с API VK

In [ ]:
token = '958eb5d439726565e9333aa30e50e0f937ee432e927f0dbd541c541887d919a7c56f95c04217915c32008'

Напишем функцию, которая будет находить группы по поисковому запросу при помощи метода [groups.search](https://vk.com/dev/groups.search)

In [ ]:
def search_query(q, sorting=0):

    #Параметры sort
    #0 — сортировать по умолчанию (аналогично результатам поиска в полной версии сайта);
    #1 — сортировать по скорости роста;
    #2 — сортировать по отношению дневной посещаемости к количеству пользователей;
    #3 — сортировать по отношению количества лайков к количеству пользователей;
    #4 — сортировать по отношению количества комментариев к количеству пользователей;
    #5 — сортировать по отношению количества записей в обсуждениях к количеству пользователей.
    params = {
        'q': q,
        'access_token': token,
        'v':'5.131',
        'sort': sorting,
        'count': 300
    }
    req = requests.get('https://api.vk.com/method/groups.search', params).json()
    # print(req)
    req = req['response']['items']
    return req

# search_query('python', 0)
res_df = pd.DataFrame(search_query('python', 0))
res_df

Получим расширенную информацию по группам при помощи метода [groups.getById](https://vk.com/dev/groups.getById)

In [ ]:
# преобразуем список всех id в строку (в таком виде принимает данные параметр fields)
groups_ids = ','.join(str(x) for x in res_df.id)
groups_ids

In [ ]:
# преобразуем список всех id в строку (в таком виде принимает данные параметр fields)

params = {
    'access_token': token,
    'v':'5.131',
    'group_ids': groups_ids,
    'fields':  'members_count,activity,description'

}
req = requests.get('https://api.vk.com/method/groups.getById', params)
# req

# print(req)

pd.DataFrame(req.json()['response']).sort_values('members_count', ascending=False)

Напишем программу, которая будет искать посты в новостной ленте по указанным запросам при помощи [newsfeed.search](https://vk.com/dev/newsfeed.search)

In [ ]:
def get_newsfeed_posts(url, tag):
    newsfeed_df = pd.DataFrame()
    params = {
        'access_token': token,
        'v':'5.131',
        'q': tag,
        'count': 200,
        'extended': 1
    }    
    while True:
        result = requests.get(url, params)
        time.sleep(0.33)
        newsfeed_df = pd.concat([newsfeed_df, pd.DataFrame(result.json()['response']['items'])])
        if 'next_from' in result.json()['response'].keys():
            params['start_from'] = result.json()['response']['next_from']
        else:
            break
    return newsfeed_df


def main_vk_newsfeed():
    print('Собираем данные из новостной ленты ВК')
    tag_list = ['skillfactory']
    url = 'https://api.vk.com/method/newsfeed.search'
    df = pd.DataFrame()
    for tag in tag_list:
        df = pd.concat([df, get_newsfeed_posts(url, tag)])
    print('Собрали данные из новостной ленты ВК')
    return df

main_vk_newsfeed()

## Напишем скрипт, который будет собирать новости с сайта Коммерсанта

In [ ]:
URL = 'https://www.kommersant.ru/search/results'
params = {
    'search_query': 'python'
}

In [ ]:
res = requests.get(URL, params)

In [ ]:
res.text

'<!DOCTYPE html>\r\n<html class="no-js" lang="ru">\r\n<head>\r\n    <title>Коммерсантъ: последние новости России и мира</title>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1" />\r\n    <meta charset="utf-8" />\r\n\r\n    <meta name="format-detection" content="telephone=no" />\r\n    <meta name="title" content="Коммерсантъ: последние новости России и мира" />\r\n    <meta name="description" content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" />\r\n    <meta name="keywords" content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" />\r\n\r\n    <meta name="yandex-verification" content="50df68945a519dbd" />\r\n\r\n    \r\n<meta name="viewport" content="width=device-width, initial-scale=

In [ ]:
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<html class="no-js" lang="ru">
<head>
<title>Коммерсантъ: последние новости России и мира</title>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="Коммерсантъ: последние новости России и мира" name="title"/>
<meta content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" name="description"/>
<meta content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" name="keywords"/>
<meta content="50df68945a519dbd" name="yandex-verification"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="324580610921010" property="fb:app_id"/>
<meta content="website" propert

In [ ]:
# собираем все теги со ссылками на полные тексты новостей
refs = soup.find_all('a', class_='uho__link')
print(len(refs))
print(refs)

26
[<a class="uho__link uho__link--overlay" href="/doc/5293079?query=python" target="_blank">Самое главное о профессии геймдизайнера</a>, <a class="uho__link" href="/doc/5293079?query=python" target="_blank"> ... # и Java, затем — <mark>Python</mark> и С++), уметь ... </a>, <a class="uho__link uho__link--overlay" href="/doc/5295067?query=python" target="_blank">Разработчиков поставили на паузу</a>, <a class="uho__link" href="/doc/5295067?query=python" target="_blank">Компания Intel приостановила работу в России, но не распустила нижегородский центр</a>, <a class="uho__link" href="/doc/5295067?query=python" target="_blank"> ... , аналитиков, разработчиков (JAVA, <mark>Python</mark>), SDET, Test Automation ... </a>, <a class="uho__link uho__link--overlay" href="/doc/5292974?query=python" target="_blank">Кадры идут учиться</a>, <a class="uho__link" href="/doc/5292974?query=python" target="_blank">Какие специалисты востребованы у работодателей</a>, <a class="uho__link" href="/doc/5292974?q

In [ ]:
# добираемся до ссылок
all_links = []

for ref in refs:
    all_links.append(ref.get('href'))

print(len(all_links))
print(all_links)

26
['/doc/5293079?query=python', '/doc/5293079?query=python', '/doc/5295067?query=python', '/doc/5295067?query=python', '/doc/5295067?query=python', '/doc/5292974?query=python', '/doc/5292974?query=python', '/doc/5292974?query=python', '/doc/5271237?query=python', '/doc/5271237?query=python', '/doc/5258915?query=python', '/doc/5258915?query=python', '/doc/5258915?query=python', '/doc/5238663?query=python', '/doc/5238663?query=python', '/doc/5238663?query=python', '/doc/5179940?query=python', '/doc/5179940?query=python', '/doc/5179940?query=python', '/doc/5171805?query=python', '/doc/5171805?query=python', '/doc/5139396?query=python', '/doc/5139396?query=python', '/doc/5139396?query=python', '/doc/5140728?query=python', '/doc/5140728?query=python']


In [ ]:
# исключаем дубли
all_links = set(all_links)
print(len(all_links))

10


In [ ]:
# формируем полноценные ссылки
all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))
print(all_full_links)

['https://www.kommersant.ru/doc/5179940?query=python', 'https://www.kommersant.ru/doc/5140728?query=python', 'https://www.kommersant.ru/doc/5292974?query=python', 'https://www.kommersant.ru/doc/5295067?query=python', 'https://www.kommersant.ru/doc/5293079?query=python', 'https://www.kommersant.ru/doc/5139396?query=python', 'https://www.kommersant.ru/doc/5271237?query=python', 'https://www.kommersant.ru/doc/5171805?query=python', 'https://www.kommersant.ru/doc/5258915?query=python', 'https://www.kommersant.ru/doc/5238663?query=python']


In [ ]:
# объединим все в одну функцию
def get_all_links(query):
    url = 'https://www.kommersant.ru/search/results'
    params = {
        'search_query': query,
    }
    res = requests.get(URL, params)
    soup = BeautifulSoup(res.text)
    refs = soup.find_all('a', class_='uho__link')

    all_links = []
    for ref in refs:
        all_links.append(ref.get('href'))
    
    all_links = set(all_links)

    all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

    return all_full_links

all_links = get_all_links('python')
print(all_links)

['https://www.kommersant.ru/doc/5179940?query=python', 'https://www.kommersant.ru/doc/5140728?query=python', 'https://www.kommersant.ru/doc/5292974?query=python', 'https://www.kommersant.ru/doc/5295067?query=python', 'https://www.kommersant.ru/doc/5293079?query=python', 'https://www.kommersant.ru/doc/5139396?query=python', 'https://www.kommersant.ru/doc/5271237?query=python', 'https://www.kommersant.ru/doc/5171805?query=python', 'https://www.kommersant.ru/doc/5258915?query=python', 'https://www.kommersant.ru/doc/5238663?query=python']


In [ ]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
def get_all_links(query, pages):
    url = 'https://www.kommersant.ru/search/results'
    links_list = []
    params = {
        'search_query': query
    }
    for i in range(1, pages+1):
        params['page'] = i
        res = requests.get(URL, params)
        time.sleep(0.3)

        refs = soup.find_all('a', class_='uho__link')
        all_links = []

        for ref in refs:
            all_links.append(ref.get('href'))
        
        all_links = set(all_links)

        all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

        links_list += all_full_links
    return links_list

all_links = get_all_links('python', 5)
print(len(all_links))
print(all_links)

50
['https://www.kommersant.ru/doc/5179940?query=python', 'https://www.kommersant.ru/doc/5140728?query=python', 'https://www.kommersant.ru/doc/5292974?query=python', 'https://www.kommersant.ru/doc/5295067?query=python', 'https://www.kommersant.ru/doc/5293079?query=python', 'https://www.kommersant.ru/doc/5139396?query=python', 'https://www.kommersant.ru/doc/5271237?query=python', 'https://www.kommersant.ru/doc/5171805?query=python', 'https://www.kommersant.ru/doc/5258915?query=python', 'https://www.kommersant.ru/doc/5238663?query=python', 'https://www.kommersant.ru/doc/5179940?query=python', 'https://www.kommersant.ru/doc/5140728?query=python', 'https://www.kommersant.ru/doc/5292974?query=python', 'https://www.kommersant.ru/doc/5295067?query=python', 'https://www.kommersant.ru/doc/5293079?query=python', 'https://www.kommersant.ru/doc/5139396?query=python', 'https://www.kommersant.ru/doc/5271237?query=python', 'https://www.kommersant.ru/doc/5171805?query=python', 'https://www.kommersant.

In [ ]:
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
    print(date)
    title = soup.find('h1', class_='doc_header__name').text
    print(title)
    text = soup.find('div', class_='doc__body').text
    print(text)

Выходные данные были обрезаны до нескольких последних строк (5000).

...


























Самой востребованной вакансией в Ярославской области в 2021 году стала вакансия «продавец», включающая вариации «продавец-консультант», «кассир» и другие синонимы профессии. Соответствующее исследование провела платформа онлайн-рекрутинга hh.ru. Количество вакансий «продавец» в регионе по итогам прошедшего года приблизилось к 5 тыс., заняв долю 6,8% от общего числа доступных вакансий в регионе.






Выйти из полноэкранного режима








Развернуть на весь экран













Фото: Иван Водопьянов, Коммерсантъ








Фото: Иван Водопьянов, Коммерсантъ




Водители — вторая по востребованности профессия. Количество вакансий на платформе — 3,1 тыс., доля от общего количества по региону — 4,6%. Третье место по востребованности заняли вакансии «менеджер по продажам» — 2,6 тыс. штук, эквивалентных 3,8% от общего количества по Ярославской области.
Самой востребованной профессии «продавец» ярос

In [ ]:
# запишем данные в датафрейм
kom_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
    title = soup.find('h1', class_='doc_header__name').text.strip()
    text = soup.find('div', class_='doc__body').text.strip().replace('\n', '')
    row = {'date': date, 'title': title, 'text': text}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news

,date,title,text
0,2022-01-21 13:23:20+03:00,Требуются продавцы. Срочно!,3K\r \r 1 мин.\r ...
0,2021-12-20 17:37:05+03:00,Нижегородские школьники смогут бесплатно пройт...,395\r \r 1 мин.\r ...
0,2022-04-03 15:28:40+03:00,Кадры идут учиться,915\r \r 3 мин.\r ...
0,2022-04-06 21:09:05+03:00,Разработчиков поставили на паузу,2K\r \r 4 мин.\r ...
0,2022-04-07 15:48:19+03:00,Самое главное о профессии геймдизайнера,4K\r \r 5 мин.\r ...
0,2021-12-28 00:01:00+03:00,Битва за кадры,522\r \r 8 мин.\r ...
0,2022-03-23 12:23:11+03:00,Расширение штата по всем направлениям!,690\r \r 1 мин.\r ...
0,2022-01-18 10:14:53+03:00,Два центра цифрового образования создадут в Уд...,377\r \r 1 мин.\r ...
0,2022-03-16 00:03:00+03:00,EPAM System и JetBrains приостанавливают деяте...,5K\r \r 5 мин.\r ...
0,2022-03-01 17:23:56+03:00,Porsche Macan в третьем поколении,24\r \r 3 мин.\r ...


In [ ]:
# обернем в функцию 
def get_kom_news(links):
    kom_news = pd.DataFrame()
    for link in all_links:
        soup = BeautifulSoup(requests.get(link).text)
        time.sleep(0.3)
        date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
        title = soup.find('h1', class_='doc_header__name').text.strip()
        text = soup.find('div', class_='doc__body').text.strip().replace('\n', '')
        row = {'date': date, 'title': title, 'text': text}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
    return kom_news.reset_index(drop=True)

get_kom_news(all_links)

,date,title,text
0,2022-01-21 13:23:20+03:00,Требуются продавцы. Срочно!,3K\r \r 1 мин.\r ...
1,2021-12-20 17:37:05+03:00,Нижегородские школьники смогут бесплатно пройт...,395\r \r 1 мин.\r ...
2,2022-04-03 15:28:40+03:00,Кадры идут учиться,915\r \r 3 мин.\r ...
3,2022-04-06 21:09:05+03:00,Разработчиков поставили на паузу,2K\r \r 4 мин.\r ...
4,2022-04-07 15:48:19+03:00,Самое главное о профессии геймдизайнера,4K\r \r 5 мин.\r ...
5,2021-12-28 00:01:00+03:00,Битва за кадры,522\r \r 8 мин.\r ...
6,2022-03-23 12:23:11+03:00,Расширение штата по всем направлениям!,690\r \r 1 мин.\r ...
7,2022-01-18 10:14:53+03:00,Два центра цифрового образования создадут в Уд...,377\r \r 1 мин.\r ...
8,2022-03-16 00:03:00+03:00,EPAM System и JetBrains приостанавливают деяте...,5K\r \r 5 мин.\r ...
9,2022-03-01 17:23:56+03:00,Porsche Macan в третьем поколении,24\r \r 3 мин.\r ...


# Спасибо за внимание! Буду рад ответить на ваши вопросы
Форма ОС: https://forms.gle/y8xaFwJqtbFSjUeG8